In [1]:
using PlotlyJS
using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")

WebIO._IJuliaInit()

change_type (generic function with 1 method)

In [2]:
folder_path = joinpath("..","output", "solutions_v2.3")
solution_folders = ["n_228", "n_259", "n_289"]
# solution_folders = ["n_228"]

keys = [:demand, :generation, :storage, :reserve, :scalar]
s_uc = [parquet_to_solution("s_uc", joinpath(folder_path, s)) for s in solution_folders]
s_ed = [parquet_to_solution("s_ed", joinpath(folder_path, s)) for s in solution_folders]
s_uc = NamedTuple(k => vcat([s[k] for s in s_uc]...) for k in keys)
s_ed = NamedTuple(k => vcat([s[k] for s in s_ed]...) for k in keys)


reading...
../output/solutions_v2.3/n_228/s_uc_demand.parquet
../output/solutions_v2.3/n_228/s_uc_generation.parquet
../output/solutions_v2.3/n_228/s_uc_storage.parquet
../output/solutions_v2.3/n_228/s_uc_reserve.parquet
../output/solutions_v2.3/n_228/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.3/n_259/s_uc_demand.parquet
../output/solutions_v2.3/n_259/s_uc_generation.parquet
../output/solutions_v2.3/n_259/s_uc_storage.parquet
../output/solutions_v2.3/n_259/s_uc_reserve.parquet
../output/solutions_v2.3/n_259/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.3/n_289/s_uc_demand.parquet
../output/solutions_v2.3/n_289/s_uc_generation.parquet
../output/solutions_v2.3/n_289/s_uc_storage.parquet
../output/solutions_v2.3/n_289/s_uc_reserve.parquet
../output/solutions_v2.3/n_289/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.3/n_228/s_ed_demand.parquet
../output/solutions_v2.3/n_228/s_ed_generation.parquet
../output/solutions_v2.3/n_228/s_ed_storage

(demand = 100800×8 DataFrame
    Row │ hour   demand_MW  r_id     resource  LOL_MW    iteration  day    con ⋯
        │ Int64  Float64    Missing  String    Float64?  Symbol     Int64  Sym ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │  5473    2469.78  missing  total          0.0  demand_86    228  bas ⋯
      2 │  5474    2290.92  missing  total          0.0  demand_86    228  bas
      3 │  5475    2149.84  missing  total          0.0  demand_86    228  bas
      4 │  5476    2046.69  missing  total          0.0  demand_86    228  bas
      5 │  5477    2327.79  missing  total          0.0  demand_86    228  bas ⋯
      6 │  5478    2326.68  missing  total          0.0  demand_86    228  bas
      7 │  5479    2770.45  missing  total          0.0  demand_86    228  bas
      8 │  5480    3166.35  missing  total          0.0  demand_86    228  bas
      9 │  5481    3771.45  missing  total          0.0  demand_86    228  bas ⋯
     10 │  

In [3]:
thres =.001 # 1 Watt
f_LOL(x,y) = 
    (LLD_h=count(x.>thres),
    # LOLP=count(x.>thres)/length(y)*100,
    ENS_MWh = sum(x),
    # LOL_percentage = sum(x)/(sum(y) + sum(x))*100,
    Demand_MWh = (sum(y) + sum(x)),
    )
f_CUR(x,y) =     
    (CURD_h=count(x.>thres),
    # CURP=count(x.>thres)/length(y)*100,
    CUR_MWh = sum(x),
    # CUR_percentage = sum(x)/(sum(y) + sum(x))*100,
    RES_production_MWh = (sum(y) + sum(x)),
    )

f_CUR (generic function with 1 method)

In [4]:
parse_configuration_to_mu(x) = !isnothing(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))) ? parse(Float64, replace(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))[1], "_" => ".")) : missing


parse_configuration_to_mu (generic function with 1 method)

In [5]:
group_by = [:configuration, :day]
group_by_big = [:iteration, :configuration, :day]

filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic", "net_generation"]).(s_ed.generation.resource)

gdf_LOL = combine(groupby(s_ed.demand, group_by_big), [:LOL_MW, :demand_MW] => ((x,y)->f_LOL(x,y)) => AsTable)
gdf_CUR = combine(groupby(s_ed.generation[filter,:], group_by_big), [:curtailment_MW, :production_MW] =>((x,y) -> f_CUR(x,y))=> AsTable)
g_KPI = outerjoin(gdf_LOL, gdf_CUR, on = group_by_big)
# if write
#     XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))
#     XLSX.writetable(joinpath(folder_path,"gdf_LOL_CUR.xlsx"), "gdf_LOL_CUR" =>  change_type(gdf_LOL_CUR, Symbol, string))
# end
;

In [6]:
# folowing leftjoin will repeat values right values for "iteration"
s_ed_scalar = s_ed.scalar[:, Not(:termination_status)]
leftjoin!(
    s_ed_scalar, 
    rename(s_uc.scalar[:, Not(:termination_status)], :objective_value =>:objective_value_uc),
    on = setdiff(propertynames(s_ed_scalar), [:objective_value, :iteration]))
    
# realtive difference with respect to s_uc
s_ed_scalar.Δobjective_value = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc
s_ed_scalar.Δobjective_value_relative = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc)./s_ed_scalar.objective_value_uc

# relative difference with respect to ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
ref_configuration = s_ed.scalar[s_ed.scalar.configuration .== ref_configuration, [:objective_value, :iteration, :day]]

leftjoin!(s_ed_scalar, 
    rename(ref_configuration, :objective_value => :objective_value_ref_conf), 
    on = [:day, :iteration])

s_ed_scalar.Δobjective_value_ref_conf = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf
s_ed_scalar.Δobjective_value_relative_ref_conf = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf)./s_ed_scalar.objective_value_ref_conf

leftjoin!(g_KPI, s_ed_scalar, on = group_by_big)
;
# # s_ed_scalar = leftjoin(left, right, on = setdiff(propertynames(left), [:objective_value, :iteration]))

In [7]:
LOL = combine(groupby(gdf_LOL, group_by), [:LLD_h, :ENS_MWh] => ((x,y)->(LOLE = mean(x), EENS = mean(y))) => AsTable)
CUR = combine(groupby(gdf_CUR, group_by), [:CURD_h, :CUR_MWh] => ((x,y)->(CURE = mean(x), ECUR = mean(y))) => AsTable)

KPI = outerjoin(
    LOL, 
    CUR,
    combine(groupby(g_KPI, group_by), [:objective_value, :Δobjective_value_relative_ref_conf] .=> mean .=> [:EOV, :EΔOV]),
    on=[:configuration, :day])

transform!(KPI, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
sort!(KPI, :mu)

transform!(g_KPI, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
sort!(g_KPI, :mu)
;

In [8]:
# plot(s_ed_scalar, y = :objective_value_relative, x = :ENS_MWh, mode = "markers", marker=attr(showscale=true, coloraxis="coloraxis", color=:configuration))
plot(g_KPI, y = :Δobjective_value_ref_conf , x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration)#marker=attr(showscale=true, coloraxis="coloraxis", color=:configuration))

┌ Warning: attempting to remove probably stale pidfile
│   path = "/home/ubuntu/.jlassetregistry.lock"
└ @ Pidfile ~/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260


data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgro

In [9]:
to_plot = g_KPI[abs.(g_KPI.Δobjective_value_relative_ref_conf) .>0.001,:]
plot(to_plot, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, yaxis, yaxis2, and yaxis3"

In [10]:
p1 = plot(KPI, x = :mu, y = :EΔOV, color = :day)
p2 = plot(KPI, x = :mu, y = :EENS, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [11]:
plot(g_KPI[g_KPI.day .== 228,:], x = :mu, y = :ENS_MWh, kind = "box",  Layout(boxmode="group"))

data: [
  "box with fields type, x, xaxis, y, and yaxis"
]

layout: "layout with fields boxmode, legend, margin, template, xaxis, and yaxis"

In [12]:
# parse_configuration_to_mu(x) = !isnothing(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))) ? parse(Float64, replace(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))[1], "_" => ".")) : 1.1
# to_plot = transform(s_ed_scalar, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(to_plot, :mu)
plot(g_KPI, y = :ENS_MWh, facet_col = :day, x = :mu, mode = "markers", color = :iteration)

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgro

In [13]:
plot(g_KPI, x = :ENS_MWh, color =:configuration, facet_col = :mu, facet_row = :day, kind ="histogram")

data: [
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, showlegend, type, x, xaxis, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis10, xaxis11, xaxis12, xaxis13, xaxis14, xaxis15

In [14]:
# plot(gdf_LOL_CUR, x = :CUR_MWh, y = :ENS_MWh, facet_col = :configuration, text = :iteration, color= :day, mode="markers")

In [15]:
# plot(LOL_CUR, x = :LOLE, y = :EENS, facet_col = :day, group = :configuration, marker_color = :EOV_M, mode = "markers", marker=attr(showscale=true))
# configs
plot(KPI, x = :ECUR, y = :EENS, facet_col = :day, mode = "markers", text = :day, color=:configuration) # marker=attr(showscale=true, coloraxis="coloraxis", color=:day

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  

In [16]:
# plot(LOL_CUR, x = :EOV, y = :EENS, text = :configuration, color = :configuration, mode = "markers")
plot(KPI, x = :EOV, y = :EENS, color = :configuration, mode = "markers", facet_col = :day)


data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgro

In [17]:
supply_uc, demand_uc = calculate_supply_demand(s_uc, union([:hour, :resource], group_by))
supply_ed, demand_ed = calculate_supply_demand(s_ed, union([:hour, :resource, :iteration], group_by))
commit_uc = combine(groupby(s_uc.generation, [:resource, :configuration, :day, :hour]), :commit => sum => :commit)
commit_ed = combine(groupby(s_ed.generation, [:resource, :configuration, :day, :iteration, :hour]), :commit => sum => :commit)
;

### Checking dispatch

In [18]:
day_ = 228
iteration_ = :demand_86

:demand_86

In [19]:
# Checking solutions are different
scalar = s_ed_scalar[(s_ed_scalar.day .== day_) .& (s_ed_scalar.iteration .== iteration_),:]
(scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_1_dn_1, :objective_value] - scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]) / scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]

1×1 Matrix{Float64}:
 -0.9886223252438239

In [20]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [21]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [22]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [23]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [24]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [25]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]


data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"